# Analyse et Modélisation des K-Types INOSYS

Ce notebook présente la démarche de modélisation pour la classification des exploitations agricoles en K-Types.

## Objectifs :
1. **Apprentissage Non Supervisé** : Identifier des segments naturels d'exploitations via le clustering (K-Means).
2. **Apprentissage Supervisé** : Entraîner des modèles (Random Forest, Gradient Boosting) pour prédire le K-Type à partir des variables d'entrée.
3. **Optimisation** : Recherche des meilleurs hyperparamètres via GridSearchCV.
4. **Export** : Sauvegarder le meilleur modèle pour une utilisation en production (Backend).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score

import warnings
warnings.filterwarnings('ignore')

# Chargement des données
df = pd.read_csv('farms_dataset.csv')
print(f"Dataset chargé : {df.shape[0]} lignes, {df.shape[1]} colonnes")
df.head()

## 1. Préparation des données

Nous séparons les variables explicatives (Features) de la cible (K-Type). Les variables incluent la SAU, l'UGB, la filière, etc.

In [ ]:
# Nettoyage : suppression des K-Types trop rares (pour la stabilité du modèle)
counts = df['ktype'].value_counts()
rare_ktypes = counts[counts < 2].index
df = df[~df['ktype'].isin(rare_ktypes)]

X = df.drop(['ktype'], axis=1)
y = df['ktype']

# Définition des colonnes numériques et catégorielles
numeric_features = ['sau', 'umo', 'ugb', 'nb_vl', 'surface_sfp', 'surface_herbe_pp', 'surface_herbe_pt', 'surface_culture']
categorical_features = ['filiere']

# Prétraitement : Standardisation pour le numérique, OneHot pour le catégoriel
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

print("Prétraitement configuré.")

## 2. Apprentissage Non Supervisé (Clustering)

Nous utilisons K-Means pour voir si les exploitations se regroupent naturellement de manière cohérente avec les filières ou les K-Types.

In [ ]:
X_processed = preprocessor.fit_transform(X)

# On teste 7 clusters (correspondant aux 7 filières principales)
kmeans = KMeans(n_clusters=7, random_state=42)
clusters = kmeans.fit_predict(X_processed)

df['cluster'] = clusters

# Visualisation de la répartition des K-Types dans les clusters (Extrait)
pivot_table = pd.crosstab(df['cluster'], df['ktype']).head(10)
plt.figure(figsize=(12, 6))
sns.heatmap(pivot_table, annot=True, cmap='YlGnBu')
plt.title("Répartition des K-Types par Cluster (Extrait)")
plt.show()

## 3. Apprentissage Supervisé et Optimisation

Nous comparons deux modèles : **Random Forest** et **Gradient Boosting**. Nous utilisons GridSearchCV pour trouver les meilleurs paramètres.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Pipelines pour les modèles
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Grille de recherche simplifiée pour RandomForest
param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 15]
}

grid_search = GridSearchCV(rf_pipeline, param_grid, cv=3, scoring='accuracy', n_jobs=1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
print(f"Meilleurs paramètres : {grid_search.best_params_}")
print(f"Score de validation croisée : {grid_search.best_score_:.4f}")

## 4. Évaluation Finale

Évaluation du modèle sur le jeu de test non vu lors de l'entraînement.

In [ ]:
y_pred = best_model.predict(X_test)

print("--- Rapport de Classification ---")
# Note: On limite l'affichage aux premières classes si trop nombreuses
print(classification_report(y_test, y_pred))

# Sauvegarde du modèle
joblib.dump(best_model, 'model_ktype_optimized.pkl')
print("Modèle optimisé sauvegardé sous 'model_ktype_optimized.pkl'")

## Conclusion

Le modèle est prêt à être intégré dans le Backend. Il utilise les variables structurelles (SAU, UGB, etc.) pour prédire avec précision le K-Type correspondant au référentiel INOSYS.